# **Roadmap**

1. Install the TensorFlow Object Detection API
2. Setup folder structure
3. Generate the TFRecord files required for training
4. Edit the model pipeline config file and download the pre-trained model checkpoint
5. Train and evaluate the model

⚠️ This notebook is meant to be run in Google Colab for training in order to use GPU capacity.

In [1]:
# Check GPU setup

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 29 10:01:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Check RAM setup

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# **Import libraries**

In [3]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
import json
import shutil
print(tf.__version__)

2.8.2


# 1. Create customTF2, training and data folders in your google drive

Create a folder named ***customTF2***.

Create another folder named ***training*** inside the ***customTF2*** folder
(***training*** folder is where the checkpoints will be saved during training)

Create another folder named ***data*** inside the ***customTF2*** folder.

# 2. Mount drive and link your folder

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/Othercomputers/My_MacBook_Pro_Machine_Learning/ /MacBook
!ls /MacBook

Mounted at /content/gdrive
customTF2  Final_Dataset_Split	notebooks


# 3. Clone the tensorflow models git repository & Install TensorFlow Object Detection API

In [ ]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
# Testing the model builder
!python object_detection/builders/model_builder_tf2_test.py

# 4. Train / Test split

We will use the following repo and script (https://github.com/akarazniewicz/cocosplit) to generate a train & test split from the annotations

In [ ]:
# Command to split 80%

⬛️ python cocosplit.py --having-annotations -s 0.8 '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/final_annotations_coco_V6.json' '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/train.json' '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/test.json'

We then need to generate the train & test data folders accordingly

In [ ]:
# Function to generate the train & test folders

def generate_train_test_folders(json_file,path_to_original_data,path_to_new_split_data,name_of_split):
    with open(json_file) as f:
        json_annotations = json.load(f)
    
    for image in json_annotations['images']:
        path_image = image['file_name']
        os.makedirs(os.path.dirname(f"{path_to_new_split_data}/{name_of_split}/{path_image}"), exist_ok=True)
        shutil.copy2(f"{path_to_original_data}/{path_image}", f"{path_to_new_split_data}/{name_of_split}/{path_image}")
    
    return f"Completed transfer of {name_of_split}"


In [ ]:
# Test folder
generate_train_test_folders('/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/test.json', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset_Split', 'test')

'Completed transfer of test'

In [ ]:
# Train folder
generate_train_test_folders('/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/train.json', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset_Split', 'train')

'Completed transfer of train'

# 5. Convert Train & Test to TFRecord

In [ ]:
# CLI command using the fiftyone library to do this

fiftyone convert \
            --input-dir data/Final_Dataset/test.json \
            --input-type fiftyone.types.COCODetectionDataset \
            --output-dir data/Final_Dataset/TFRecord \
            --output-type fiftyone.types.TFObjectDetectionDataset

In [ ]:
# read the JSON to get the path
# iterate over and copy the file to new folder